In [4]:
%reload_ext autoreload
%autoreload 2

from ryanair import Ryanair
import pandas as pd

import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

my_airport_from = 'GOT'
my_airport_to = 'EDI'

api = Ryanair()


In [5]:
%%script false --no-raise-error
api = Ryanair()
res = api.get_flight_schedules(my_airport_from)
res[my_airport_to]
my_airport_to in res
# create a dataframe from the response
df = pd.DataFrame(res).T

In [ ]:
response = api.get_available_flights(my_airport_from, my_airport_to)
print(f"There are {len(response)} days with available flights from {my_airport_from}  between {response[0]} and {response[-1]}")
response.k

In [6]:
response = api.get_all_flights(
    date_out='2023-07-25',
    # destination=my_airport_to,
    destination=my_airport_from,
    # origin_airport=my_airport_from
    origin_airport=my_airport_to
    )


In [16]:
response[0]
# get keys of this named tuple
response[0]._fields

('departureTime_local',
 'departureTime_utc',
 'arrivalTime_local',
 'arrivalTime_utc',
 'flightNumber',
 'operatedBy',
 'duration',
 'actualFare',
 'publishedFare',
 'hasDiscount',
 'discountInPercent',
 'hasPromoDiscount',
 'hasBogof',
 'infantsLeft',
 'faresLeft',
 'currency',
 'origin',
 'originFull',
 'destination',
 'destinationFull')

In [32]:
from ryanair.types import FlightV2
from ryanair.airport_utils import get_airport_by_iata

In [40]:
get_airport_by_iata('GOT')

'Gothenburg-Landvetter Airport, Gothenburg'

In [45]:
response

[FlightV2(departureTime_local='2023-07-25T22:35:00.000', departureTime_utc='2023-07-25T20:35:00.000Z', arrivalTime_local='2023-07-25T23:30:00.000', arrivalTime_utc='2023-07-25T22:30:00.000Z', flightNumber='RK 5658', operatedBy='Ryanair UK', duration='01:55', actualFare=295.09, publishedFare=339.0, hasDiscount=True, discountInPercent=0, hasPromoDiscount=False, hasBogof=False, infantsLeft=17, faresLeft=2, currency='SEK', origin='GOT', originFull='Gothenburg-Landvetter Airport, Gothenburg', destination='EDI', destinationFull='Edinburgh Airport, Edinburgh'),
 FlightV2(departureTime_local='2023-07-26T23:00:00.000', departureTime_utc='2023-07-26T21:00:00.000Z', arrivalTime_local='2023-07-26T23:55:00.000', arrivalTime_utc='2023-07-26T22:55:00.000Z', flightNumber='FR 5658', operatedBy='', duration='01:55', actualFare=290.7, publishedFare=339.0, hasDiscount=True, discountInPercent=0, hasPromoDiscount=False, hasBogof=False, infantsLeft=18, faresLeft=5, currency='SEK', origin='GOT', originFull='G

In [ ]:

list_of_flights = []
currency = response['currency']  # the currency will use the origin airport if allowable otherwise defailt to locale

try:
    assert len(response['trips']) == 1
except AssertionError:
    logger.warning(f"There are multiple ({len(response['trips'])}) trips in the response, this is not expected - ignoring")
    logger.critical("STOOOOOPPPP!")
    pass

for this_date in response['trips'][0]['dates']:
    for this_flight in this_date['flights']:
        flight_dict = {}

        # currency
        flight_dict['currency'] = currency

        # faresLeft
        if 'faresLeft' in this_flight:
            flight_dict['faresLeft'] = this_flight['faresLeft']  # NB: '-1' appears to suggest it doesnt know, ie lots! (not overbooked i dont think)
            if flight_dict['faresLeft'] == -1:
                flight_dict['faresLeft'] = float('inf')
        else:
            flight_dict['faresLeft'] = None

        # flightKey
        if 'flightKey' in this_flight:
            # flight_dict['flightKey'] = this_flight['flightKey'] # can ignore for now as data is duplicated
            pass

        # infantsLeft
        if 'infantsLeft' in this_flight:
            flight_dict['infantsLeft'] = this_flight['infantsLeft']
            if flight_dict['infantsLeft'] == -1:
                flight_dict['infantsLeft'] = float('inf')
        else:
            flight_dict['infantsLeft'] = None

        # regularFare
        if 'regularFare' in this_flight:
            try:
                assert 'fares' in this_flight['regularFare'] and len(this_flight['regularFare']['fares']) == 1
                for this_fare in this_flight['regularFare']['fares']:
                    assert this_fare['type'] == 'ADT'
                    flight_dict['actualFare'] = this_fare['amount']
                    flight_dict['publishedFare'] = this_fare['publishedFare']
                    # flight_dict['price'] = this_fare['count'] # ignore
                    flight_dict['hasDiscount'] = this_fare['hasDiscount']
                    flight_dict['discountInPercent'] = this_fare['discountInPercent']
                    flight_dict['hasPromoDiscount'] = this_fare['hasPromoDiscount']
                    flight_dict['hasBogof'] = this_fare['hasBogof']
            except AssertionError:
                logger.warning(f"Unexpected regularFare structure - ignoring entire API call")
                continue

        # operatedBy
        if 'operatedBy' in this_flight:
            flight_dict['operatedBy'] = this_flight['operatedBy']

        # segments
        if 'segments' in this_flight:
            try:
                assert len(this_flight['segments']) == 1
                this_segment = this_flight['segments'][0]

                # flightNumber
                if 'flightNumber' in this_segment:
                    flight_dict['flightNumber'] = this_segment['flightNumber']

                # time
                if 'time' in this_segment:
                    assert len(this_segment['time']) == 2
                    flight_dict['departureTime_local'] = this_segment['time'][0]
                    flight_dict['arrivalTime_local'] = this_segment['time'][1]
                    # convert to datetime without using pandas
                    # flight_dict['departureTime_local'] = datetime.strptime(flight_dict['departureTime_local'], '%H:%M')
                    # flight_dict['arrivalTime_local'] = datetime.strptime(flight_dict['arrivalTime_local'], '%H:%M')

                # timeUTC
                if 'timeUTC' in this_segment:
                    assert len(this_segment['timeUTC']) == 2
                    flight_dict['departureTime_utc'] = this_segment['timeUTC'][0]
                    flight_dict['arrivalTime_utc'] = this_segment['timeUTC'][1]
                    # convert to datetime
                    # flight_dict['departureTime_utc'] = pd.to_datetime(flight_dict['departureTime_utc'])
                    # flight_dict['arrivalTime_utc'] = pd.to_datetime(flight_dict['arrivalTime_utc'])

                # duration
                flight_dict['duration'] = this_segment['duration'] if 'duration' in this_flight else None

                # origin
                if 'origin' in this_segment:
                    flight_dict['origin'] = this_segment['origin']
                    flight_dict['originFull'] = get_airport_by_iata(this_segment['origin'])
                else:
                    logger.warning(f"Unexpected origin structure - ignoring this flight")
                    logger.critical(this_flight.keys())
                    continue

                # destination
                if 'destination' in this_segment:
                    flight_dict['destination'] = this_segment['destination']
                    flight_dict['destinationFull'] = get_airport_by_iata(this_segment['destination'])
                else:
                    logger.warning(f"Unexpected destination structure - ignoring this flight")
                    continue

            except AssertionError:
                logger.warning(f"Unexpected segments structure - ignoring entire API call")
                continue
        
        print(flight_dict)
        list_of_flights.append(FlightV2(**flight_dict))
# list_of_flights

In [ ]:
# what is the top level of  the res dict
print(f"Top level of the response is {response.keys()}")
# what is the second level of the res dict
print(f"Second level of the response is")
for key in response.keys():
    if isinstance(response[key], list):
        for item in response[key]:
            print(item.keys())

# what is the third level of the res dict
print(f"Third level of the response is")
for key in response.keys():
    if isinstance(response[key], list):
        for item in response[key]:
            for key2 in item.keys():
                if isinstance(item[key2], list):
                    for item2 in item[key2]:
                        print(item2.keys())


# what is the fourth level of the res dict
print(f"Fourth level of the response is")
for key in response.keys():
    if isinstance(response[key], list):
        for item in response[key]:
            for key2 in item.keys():
                if isinstance(item[key2], list):
                    for item2 in item[key2]:
                        for key3 in item2.keys():
                            if isinstance(item2[key3], list):
                                for item3 in item2[key3]:
                                    print(item3.keys())

In [ ]:
# example dict
example_dict = {
    'a': 1,
    'b': 2,
    'c': 3}


x = example_dict['c'] if 'c' in example_dict else 2
x

In [ ]:
flight_dict['duration'] = this_flight['duration'] if 'duration' in this_flight else None